In [1]:
import pandas as pd

from pandas_datareader import data
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm

In [2]:
fig_size = (12, 4)
start_date = '1900-01-02'
today = datetime.today().strftime('%Y-%m-%d')
one_year = (datetime.today()-relativedelta(years=1)).strftime('%Y-%m-%d')
five_years = (datetime.today()-relativedelta(years=5)).strftime('%Y-%m-%d')
ten_years = (datetime.today()-relativedelta(years=10)).strftime('%Y-%m-%d')

In [3]:
def get_dma_analysis(ticker, start_date=start_date, end_date=datetime.today().strftime('%Y-%m-%d'), ma_period=200):
    df = data.DataReader(ticker, 'yahoo', start_date, end_date).reset_index()
    df['DMA'] = df['Close'].rolling(ma_period).mean()
    df['PCH_DMA'] = 100*(df['Close']-df['DMA'])/df['DMA']
    df['PCH_DMA_MEAN'] = df['PCH_DMA'].mean()
    df['PCH_DMA_MEDIAN'] = df['PCH_DMA'].median()
    df = df.round(3)
#     print('Start', df[['Date', 'Close']].set_index('Date').head(1).transpose())
#     print('\nEnd', df[['Date', 'Close', 'DMA', 'PCH_DMA', 'PCH_DMA_MEAN', 'PCH_DMA_MEDIAN']].set_index('Date').tail(1).transpose())
    return df
        
def plot_dma_analysis(df, start_date=start_date, end_date=datetime.today().strftime('%Y-%m-%d'), title=''):
    df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].plot(
        x='Date', y=['Close', 'DMA'], style = ['-', '--'],
        figsize=fig_size, grid=True, title=title)
    df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].plot(
        x='Date', y=['PCH_DMA', 'PCH_DMA_MEAN', 'PCH_DMA_MEDIAN'], style=['-', '--', '--'],
        figsize=fig_size, grid=True, title=title)
    
def plot_dma_hist(df, bins=100, title=''):
    df.plot(
        x='Date', y=['PCH_DMA'], kind='hist', bins=bins,
        figsize=fig_size, grid=True, title=title)
    
def get_dma_percentile(df, bins=100):
    current_pch_dma_rank_percentile = round(100*df['PCH_DMA'].dropna().rank(axis=0).tail(1).values[0]/len(df['PCH_DMA'].dropna()), 2)
#     print('Current PCH DMA percentile:', current_pch_dma_rank_percentile)
    return(current_pch_dma_rank_percentile)

In [4]:
# tickers = ['AMZN', 'AAPL', 'FB', 'GOOGL', 'MSFT', 'UBER', 'XOM', 'BA', 'DAL', 'UAL', 'AAL', 'SMH', 'IVV',
#            'VNQ', 'VCSH', '^GSPC', 'ES=F', 'QQQ', 'SHV', '^VIX', '^TNX', 'CL=F', '^HSI', '^NSEI', 'MCHI',
#            'INDA', 'INR=X', 'GLD', 'BTC-USD', 'BRK-B', 'V', 'WMT', 'PG', 'JNJ', 'TSLA', 'PG', 'MA', 'UNH',
#            'HD', 'VZ', 'JPM', 'NVDA', 'AXP', 'SQ', 'CVS', 'GE', 'DIS', 'CRM', 'NFLX', 'BAC', 'KO', 'INTC', 'T', 'CVX', 'IBM', 'LMT', 'SBUX', 'AMD', 'MU', 'SPOT',
#            'TWTR', 'LYFT', 'BABA', 'TCEHY', 'TSM', 'XLK', 'XLV', 'XLC', 'XLY', 'XLF', 'XLI', 'XLP', 'XLU',
#            'XLE', 'XLRE', 'XLB', 'VGSH', 'BSV', 'BIV', 'BLV', 'BND']

tickers = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'TSLA', 'V', 'WMT', 'JNJ', 'JPM', 'PG', 'NVDA', 'MA', 'UNH',
           'HD', 'DIS', 'VZ', 'CRM', 'BAC', 'KO', 'PYPL', 'CMCSA', 'ADBE', 'BRK-A', 'NFLX', 'NKE', 'PFE',
           'MRK', 'T', 'PEP', 'ABT', 'INTC', 'TMO', 'ABBV', 'CSCO', 'COST', 'ORCL', 'QCOM', 'CVX', 'TMUS',
           'ACN', 'DHR', 'MCD', 'XOM', 'AVGO', 'MDT', 'NEE', 'TXN', 'HON', 'UPS', 'BMY', 'LLY', 'UNP', 'LIN',
           'CHTR', 'AMGN', 'ZM', 'PM', 'SBUX', 'LOW', 'BA', 'C', 'RTX', 'WFC', 'MS', 'IBM', 'AMT', 'LMT',
           'BLK', 'AMD', 'NOW', 'MMM', 'CAT', 'INTU', 'AXP', 'FIS', 'SQ', 'SYK', 'SCHW', 'EL', 'CVS', 'TGT',
           'ISRG', 'GE', 'UBER', 'MDLZ', 'BKNG', 'SPGI', 'DE', 'ZTS', 'GS', 'ANTM', 'FISV', 'GILD', 'MO',
           'PLD', 'CI', 'ADP', 'CL', 'TJX', 'FDX', 'CCI', 'AMAT', 'BX', 'CSX', 'MU', 'DUK', 'BDX',
           'CB', 'SNAP', 'SHW', 'EQIX', 'ITW', 'SO', 'USB', 'D', 'LRCX', 'TFC', 'VMW', 'GM', 'NSC', 'ECL',
           'CME', 'APD', 'ATVI', 'MMC', 'GPN', 'ICE', 'VRTX', 'ADSK', 'REGN', 'PNC', 'PGR', 'HUM', 'EW', 'DG',
           'TEAM', 'MCO', 'NOC', 'DELL', 'WM', 'HCA', 'ADI', 'WDAY', 'NEM', 'SPOT', 'BSX', 'TWLO', 'KMB',
           'ETN', 'DD', 'EMR', 'LULU', 'MNST', 'SCCO', 'ILMN', 'LVS', 'GD', 'ROP', 'COP', 'KDP', 'EPD',
           'VEEV', 'PINS', 'MET', 'DOCU', 'AEP', 'EXC', 'DOW', 'CTSH', 'CVNA', 'LHX', 'DLR', 'IDXX', 'KHC',
           'BAX', 'PSA', 'TTD', 'ROST', 'MAR', 'MRNA', 'COF', 'APH', 'CTAS', 'SRE', 'BIIB', 'GIS', 'KLAC',
           'INFO', 'TEL', 'CNC', 'XEL', 'CMG', 'SYY', 'TWTR', 'EA', 'ALGN', 'CSGP', 'MTCH', 'TT', 'PH', 'A',
           'F', 'CMI', 'MCHP', 'TRV', 'STZ', 'PPG', 'EBAY', 'BK', 'XLK', 'XLV', 'XLC', 'XLY', 'XLF', 'XLI',
           'XLP', 'XLU', 'XLE', 'XLRE', 'XLB', 'VCSH', 'VGSH', 'BSV', 'BIV', 'BLV', 'BND', 'QQQ', 'IVV', 'SHV',
           'CL=F', 'MCHI', 'INDA', 'GLD', 'BTC-USD', 'DAL', 'UAL', 'AAL', 'SMH']

In [5]:
a, b, c, d, e, f = ([] for i in range(6))

for ticker in tqdm(tickers):
    try:
        df = get_dma_analysis(ticker=ticker)
        a.append(get_dma_percentile(df))
        b.append(df[['PCH_DMA']].tail(1).values[0][0])
        c.append(df[['PCH_DMA_MEAN']].tail(1).values[0][0])
        d.append(df[['PCH_DMA_MEDIAN']].tail(1).values[0][0])
        e.append(df[['Close']].tail(1).values[0][0])
        f.append(df[['DMA']].tail(1).values[0][0])
    except:
        print(ticker)

In [6]:
df = pd.DataFrame()
df['ticker'] = tickers
df['percentile'] = a
df['pch'] = b
df['pch_mean'] = c
df['pch_median'] = d
df['close'] = e
df['dma'] = f

In [13]:
df.sort_values(by='percentile', ascending=True).head(15).transpose()

,229,93,172,175,31,66,107,128,185,55,148,138,155,56,122
ticker,SHV,GILD,DLR,BAX,INTC,AMT,BDX,VRTX,BIIB,AMGN,KMB,NOC,ILMN,ZM,CME
percentile,5.57,8.55,11.3,12.28,14.69,14.93,16.19,16.94,17.2,20.65,22.95,23.87,24.51,24.64,25.62
pch,-0.067,-14.572,-7.695,-9.621,-13.717,-6.852,-6.304,-15.744,-16.046,-4.869,-2.167,-4.125,-5.579,57.523,-0.809
pch_mean,0.035,8.171,6.226,3.44,5.713,6.251,4.192,6.581,8.928,8.994,4.059,4.348,9.467,87.988,6.805
pch_median,0.011,6.391,6.526,4.961,4.548,7.846,4.422,3.294,6.301,5.666,4.076,5.109,10.961,88.308,6.874
close,110.68,60.25,132.6,76.03,47.01,228.06,229.29,216.5,242.51,220.46,139.2,309.98,305.63,427.55,173.18
dma,110.754,70.527,143.654,84.123,54.484,244.835,244.717,256.955,288.861,231.743,142.283,323.316,323.69,271.421,174.593


In [14]:
df.sort_values(by='percentile', ascending=False).head(15).transpose()

,119,83,84,200,168,88,215,188,179,149,151,232,220,100,164
ticker,GM,GE,UBER,PH,DOW,DE,XLI,INFO,MAR,ETN,EMR,INDA,XLB,FDX,MET
percentile,100,100,100,100,100,99.98,99.96,99.93,99.83,99.63,99.62,99.6,99.49,99.32,99.16
pch,63.894,41.895,53.603,50.025,38.267,45.639,24.536,25.438,36.229,31.826,27.697,21.03,24.114,62.899,29.187
pch_mean,0.433,2.061,2.412,3.978,-4.324,3.875,2.062,8.088,4.804,3.64,2.603,1.713,1.951,5.217,1.375
pch_median,0.375,3.52,3.947,4.811,-2.583,4.371,3.695,8.378,6.753,4.5,3.055,2.081,3.584,4.08,3.144
close,46.46,10.45,51.26,279.86,57.73,261.51,89.65,93.62,130.91,123.25,79.95,37.22,72.09,291.68,48.67
dma,28.348,7.365,33.372,186.543,41.753,179.561,71.987,74.635,96.096,93.494,62.609,30.753,58.084,179.056,37.674


In [15]:
df.sort_values(by='pch_median', ascending=False).head(15).transpose()

,56,180,177,170,147,76,165,136,24,70,234,163,162,11,192
ticker,ZM,MRNA,TTD,CVNA,TWLO,SQ,DOCU,TEAM,NFLX,NOW,BTC-USD,PINS,VEEV,NVDA,CMG
percentile,24.64,65.66,98.95,81.79,66.48,95.49,60.62,60.7,35.75,80.33,87.35,93.24,53.89,71.96,63.55
pch,57.523,67.397,105.353,70.051,42.784,75.57,33.125,24.329,6.684,26.904,71.478,112.852,16.924,29.668,20.148
pch_mean,87.988,55.657,32.833,36.207,24.874,27.015,35.218,19.561,17.441,14.515,22.646,26.508,15.459,13.702,10.125
pch_median,88.308,48.282,35.075,30.915,28.77,27.775,26.763,19.881,17.521,15.401,15.153,14.841,14.69,13.212,12.68
close,427.55,98.56,844.1,236.72,292,202.9,217.2,213.5,482.88,514.69,19253.1,64.03,265.58,518.31,1262.82
dma,271.421,58.878,411.048,139.205,204.505,115.567,163.155,171.721,452.625,405.574,11227.7,30.082,227.139,399.722,1051.05


In [17]:
df.sort_values(by='pch_median', ascending=True).head(15).transpose()

,174,168,237,194,202,229,222,223,221,119,226,224,69,144,219
ticker,KHC,DOW,AAL,TWTR,F,SHV,VGSH,BSV,VCSH,GM,BND,BIV,AMD,NEM,XLRE
percentile,89.84,100,63.59,87.56,97.2,5.57,28.42,57.09,87.17,100,52.52,56.82,81.27,35.73,72.58
pch,11.003,38.267,10.452,25.734,44.739,-0.067,-0.103,0.262,1.447,63.894,0.784,1.273,31.898,-4.608,4.644
pch_mean,-6.586,-4.324,-0.326,0.61,1.83,0.035,0.107,0.255,0.276,0.433,0.46,0.628,5.298,2.215,1.09
pch_median,-5.555,-2.583,-1.43,-0.887,-0.406,0.011,0.039,0.128,0.272,0.375,0.696,0.934,1.462,1.557,1.617
close,33.83,57.73,14.82,45.23,9.45,110.68,62,82.86,83.03,46.46,88.22,93.27,85.07,56.87,36.8
dma,30.477,41.753,13.418,35.973,6.529,110.754,62.064,82.643,81.846,28.348,87.533,92.098,64.497,59.617,35.167
